In [66]:
import os
import requests

In [67]:
base_url = "https://api.mangadex.org"
languages = ["en"]

In [68]:
def download_chapter(chapter_id, url_to_dir, chapter_number):
    
    print(f"{base_url}/at-home/server/{chapter_id}")
    r = requests.get(f"{base_url}/at-home/server/{chapter_id}")
    r_json = r.json()

    host = r_json["baseUrl"]
    chapter_hash = r_json["chapter"]["hash"]
    data = r_json["chapter"]["data"]
    data_saver = r_json["chapter"]["dataSaver"]
    
    # Making a folder to store the images in.
    folder_path = f"{url_to_dir}/chapter-{chapter_number}"
    print(folder_path)
    
    os.makedirs(folder_path, exist_ok=True)
    
    
    for page in data:
        r = requests.get(f"{host}/data/{chapter_hash}/{page}")
        with open(f"{folder_path}/{page}", mode="wb") as f:
            f.write(r.content)

    print(f"Downloaded {len(data)} pages.")

In [69]:
def download_manga(title):
    # Searching for the id of the given manga
    res = requests.get(
        f"{base_url}/manga",
        params={"title": title}
    )
    
    # Useful data from a manga title => id, desc (en), publicationDemographic
    print(res.json()["data"][0]["id"])
    print(res.json()["data"][0]["attributes"]["title"]["en"])
    print(res.json()["data"][0]["attributes"]["description"]["en"])
    print(res.json()["data"][0]["attributes"]["publicationDemographic"])
    
    # TODO: trebace nam cover art
    
    manga_id = res.json()["data"][0]["id"]
    
    res = requests.get(
        f"{base_url}/manga/{manga_id}/aggregate",
        params={"translatedLanguage[]": languages},
    )
    
    volumes = res.json()["volumes"]
    # dict (chapter_number -> chapter_id)
    chapters = []
    
    url_to_dir = "./" + title
    os.mkdir(url_to_dir)
    
    for (volume_number, volume_data) in volumes.items():
        print(volume_number)
        print("=====================================================")
        print(volume_data)
        
        chapters = volume_data["chapters"]
        for (chapter_number, chapter_data) in chapters.items():
            chapter_id = chapter_data["id"]
            
            download_chapter(chapter_id, url_to_dir, chapter_number)
            

In [70]:
#download_manga("Berserk")

801513ba-a712-498c-8f57-cae55b38cc92
Berserk
Guts, known as the Black Swordsman, seeks sanctuary from the demonic forces attracted to him and his woman because of a demonic mark on their necks, and also vengeance against the man who branded him as an unholy sacrifice. Aided only by his titanic strength gained from a harsh childhood lived with mercenaries, a gigantic sword, and an iron prosthetic left hand, Guts must struggle against his bleak destiny, all the while fighting with a rage that might strip him of his humanity.  
  
---
- **Won the 6th Osamu Tezuka Cultural Prize Excellence Award in 2002**
seinen
none
{'volume': 'none', 'count': 14, 'chapters': {'371': {'chapter': '371', 'id': '46b88eda-73d4-4698-ba05-50d47e86b9d4', 'others': ['c9e27928-258a-4a13-841b-56997de30d50'], 'count': 2}, '370': {'chapter': '370', 'id': 'e41e2969-b048-43d7-bf39-8f8560de264f', 'others': ['9bdf19d6-20f2-4e88-b04f-7c8bfea83162'], 'count': 2}, '369': {'chapter': '369', 'id': '1de98fa9-a8ec-422d-8827-316

KeyboardInterrupt: 